# Import the Necessary Libraries

In [1]:
#IMPORT THE PANDAS AND NUMPY LIBRARIES
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')

#IMPORT THE SCIKIT-LEARN LIBRARIES
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D

#IMPORT THE VISUALIZATION LIBRARIES
import plotly.express as px
import kaleido
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#IMPORT THE SQALCHEMY LIBRARY's CREATE_ENGINE METHOD
import mysql.connector
from mysql.connector import Error

# Import data from MySQL DB

In [21]:
try:
    connection = mysql.connector.connect(host='scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
                                         database='pipedrive',
                                         user='pipedrive',
                                         password='#8LsH25%ZD', use_pure=True)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        data = pd.read_sql('SELECT * FROM clustering', connection)
        print("Connected to MySQL database... MySQL Server version on ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL database... MySQL Server version on  8.0.35
MySQL connection is closed


# Data Preprocessing

In [131]:
#CHECK THE SHAPE OF THE DATAFRAME
data.shape

(3957583, 28)

In [25]:
data

,mkt_acquisition_channel,Codice_Fiscale,person_add_time,is_archived,status,city,pipeline_id,is_lead,title,owner_name
0,NaN,NaN,NaT,1,deal,NaN,0,1,dsfsre lead,
1,,NaN,2022-06-10 15:36:56,1,deal,NaN,0,1,23099,
2,dem,NaN,2022-06-10 17:02:55,1,deal,Roma,0,1,23106,
3,SocialPaid,NaN,2022-06-10 17:57:08,1,deal,Torino,0,1,23108,
4,SocialPaid,NaN,2022-06-10 18:19:46,1,deal,Torino,0,1,23111,
...,...,...,...,...,...,...,...,...,...,...
202236,Outbound,GRBRLS62D49F205N,2024-03-12 00:00:00,2,open,Milano - Provincia,2,0,GORBANI RITA ELISABETTA,Giada Mazza
202237,Outbound,PNGWTR65C24F205N,2024-02-28 00:00:00,2,open,Milano,2,0,PANGHERZ WALTER MARIA,erick
202238,Outbound,BCCMND43C42I474X,2024-03-12 00:00:00,2,open,Milano - Paullo,2,0,BACCOLINI MIRANDA,Valeria Grassi
202239,Outbound,MRCFRZ62M17F351S,2024-03-12 00:00:00,2,open,Roma,2,0,MERCENATI FABRIZIO,Claudia Mastrota


### Removing Duplicates

In [132]:
#Drop Duplicates Rows
data.drop_duplicates(inplace=True)

### Data Imputation

In [201]:
#CHECK NULL VALUES
null_values = data.isnull().sum() / len(df) * 100
null_values[null_values > 0].sort_values(ascending=True)

division                   0.11
attribution               11.67
mkt_acquisition_channel   95.61
Codice_Fiscale            95.61
is_archived               95.61
status                    95.61
city                      95.61
pipeline_id               95.61
is_lead                   95.61
title                     95.61
owner_name                95.61
person_add_time           95.61
dtype: float64

In [134]:
#CHECK THE DATA TYPES
data.dtypes

id_intestati                                     object
codfisc                                          object
attribution                                      object
quota                                            object
titolarita                                       object
Total_IMM                                         int64
IMM_MILANO_HINTERLAND_No_small_cities             int64
IMM_MILANO_HINTERLAND_Small_cities                int64
IMM_ROMA_TERRITORIO                               int64
IMM_ROMA_CITTA                                    int64
IMM_ROMA_PROVINCIA                                int64
IMM_TORINO_TERRITORIO                             int64
IMM_TORINO_PROVINCIA                              int64
IMM_FIRENZE_TERRITORIO                            int64
IMM_FIRENZE_PROVINCIA                             int64
IMM_GENOVA_TERRITORIO                             int64
IMM_GENOVA_PROVINCIA                              int64
IMM_OTHER                                       

#### Quota Column

In [135]:
#QUOTA. FILL THE NULL VALUES OF QUOTA WITH THE MODE VALUE
data['quota'].fillna(data['quota'].mode()[0], inplace=True)
data['quota'][data['quota'] == ''] = 0

In [136]:
#Split Quota column from df dataframe into two columns by "/"
data[['numerator','denominator']] = data['quota'].str.split("/",expand=True)

In [156]:
#Convert NaNs to a number (e.g., -1 or 0) or drop the rows with NaNs
data['numerator'] = data['numerator'].fillna(0) 
#Where there is numerator values. Fill the missing values in the 'denominator' column with the values in the 'numerator' column
data['denominator'][(data['denominator'].isnull())] = data['numerator'][(data['denominator'].isnull())]
#Looking for the most frequent values in the column
data.groupby('denominator')['denominator'].count().sort_values(ascending=False).head(50)
data['denominator'][data['denominator'] == ''] = data['numerator'][data['denominator'] == '']

In [197]:
#Create a column named 'division' with the ratio of 'numerator' and 'denominator'
data['%Ownership'] = data['numerator'].astype(float) / data['denominator'].astype(float) 
#Fill na values of %Ownership column with 1.00
data['%Ownership'].fillna(1.00, inplace=True)

In [198]:
data.groupby('%Ownership')['%Ownership'].count().sort_values(ascending=False)

%Ownership
1.00        2143546
0.50        1200906
0.17         111150
0.33         102107
0.25          75118
             ...   
0.02              1
0.13              1
0.13              1
0.02              1
17777.64          1
Name: %Ownership, Length: 6702, dtype: int64

In [202]:
data[data['attribution'].isnull()]

,id_intestati,codfisc,attribution,quota,titolarita,Total_IMM,IMM_MILANO_HINTERLAND_No_small_cities,IMM_MILANO_HINTERLAND_Small_cities,IMM_ROMA_TERRITORIO,IMM_ROMA_CITTA,...,status,city,pipeline_id,is_lead,title,owner_name,numerator,denominator,division,%Ownership
25949,7c2d7907-7145-4111-bc93-f8bf3299994b,06782321001,None,1/1,Proprieta per 1/1,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.00,1.00
32414,4450616f-37b2-42f7-9a24-c4e6aa9f6591,09479010580,None,1/1,Proprieta per 1/1,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.00,1.00
41210,8b3454d3-ce84-4ba0-8a8d-5c86f5a7d67d,44044569000,None,1/2,Proprieta per 1/2,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0.50,0.50
59673,295789ea-0962-4e82-8923-682cf2de8377,BAEBNG86M28Z213I,None,1/2,Proprieta per 1/2,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0.50,0.50
59674,b2981071-779e-4a4e-9234-60bfcfeff37d,BAEHWN62B54Z213Q,None,1/2,Proprieta per 1/2,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0.50,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957561,d44687ff-1e19-4f85-a44a-bcf3f0308a81,ZZZVLR63A24B842Q,None,100/100,Proprieta per 100/100,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,100,100,1.00,1.00
3957566,c83f2efa-ab12-4579-be91-87a36695f5b4,ZZZVLR89D54H501A,None,1/1,Proprieta per 1/1,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.00,1.00
3957570,615c1497-1e9c-4107-8dcf-6aec53908186,ZZZVNT38M04C741O,None,1/2,Proprieta per 1/2,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0.50,0.50
3957573,34b074ce-6d1d-418b-ae43-a9f445c0b3fa,ZZZVNT83L63H501I,None,3/6,Proprieta per 3/6,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,3,6,0.50,0.50


#### Titolarita Column


In [177]:
data['titolarita'].fillna(data['quota'].mode()[0], inplace=True)

In [185]:
data['titolarita'] = data['titolarita'].str.replace("'", "")

In [186]:
data.groupby('titolarita')['titolarita'].count().sort_values(ascending=False)

titolarita
Proprieta                       1546473
Proprieta per 1/2                688631
Proprieta per 1/1                683934
Proprieta per 1/6                 66055
                                  63457
                                 ...   
Proprieta per 648/10000               1
Proprieta per 647022/6000000          1
Proprieta per 6443/100000             1
Proprieta per 644/1000                1
z                                     1
Name: titolarita, Length: 29624, dtype: int64

### Descriptive Analysis

In [36]:
#CHECK THE STATISTICAL SUMMARY OF THE DATAFRAME
data.describe()

,Total_IMM,IMM_MILANO_HINTERLAND_No_small_cities,IMM_MILANO_HINTERLAND_Small_cities,IMM_ROMA_TERRITORIO,IMM_ROMA_CITTA,IMM_ROMA_PROVINCIA,IMM_TORINO_TERRITORIO,IMM_TORINO_PROVINCIA,IMM_FIRENZE_TERRITORIO,IMM_FIRENZE_PROVINCIA,IMM_GENOVA_TERRITORIO,IMM_GENOVA_PROVINCIA,IMM_OTHER,person_add_time,is_lead
count,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,3957221.00,173663,173754.00
mean,2.01,0.60,0.68,0.31,0.31,0.28,0.09,0.09,0.08,0.00,0.03,0.01,0.45,2023-10-17 18:21:23.216438784,0.86
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2022-06-11 06:15:36,0.00
25%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-07-11 16:21:31,1.00
50%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-10-25 00:00:00,1.00
75%,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-01-12 00:32:42,1.00
max,74692.00,42183.00,47011.00,74692.00,74692.00,311.00,10000.00,148.00,7253.00,78.00,991.00,67.00,9816.00,2024-03-28 00:00:00,1.00
std,52.60,28.78,32.45,40.21,40.21,0.83,6.18,0.79,3.71,0.11,0.66,0.16,5.92,NaN,0.34


### Data Visualization

In [16]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [17]:
sns.histplot(data['deal_value'], bins=30, kde=True)
plt.title('Distribution of Deal Values')
plt.xlabel('Deal Value')
plt.ylabel('Frequency')
plt.show()

KeyError: 'deal_value'

In [ ]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

In [ ]:
sns.boxplot(data['deal_value'])
plt.title('Distribution of Deal Values')
plt.xlabel('Deal Value')
plt.show()

# Feature Engineering

In [ ]:
#CREATE A NEW COLUMN FOR THE DEAL VALUE
data['deal_value_log'] = np.log1p(data['deal_value'])

In [ ]:
#PLOT THE DISTRIBUT
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(data['deal_value_log'], bins=30, kde=True)
plt.title('Distribution of Log Deal Values')
plt.xlabel('Log Deal Value')
plt.ylabel('Frequency')
plt.show()

# Feature Selection - Principal Component Analysis (PCA)

In [ ]:
#SELECT THE COLUMNS FOR THE CLUSTERING
X = data[['deal_value_log']]

In [ ]:
#CHECK THE SHAPE OF THE DATAFRAME
X.shape

In [ ]:
#CHECK THE FIRST FEW ROWS OF THE DATAFRAME
X.head()

In [ ]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(X['deal_value_log'], bins=30, kde=True)
plt.title('Distribution of Log Deal Values')
plt.xlabel('Log Deal Value')
plt.ylabel('Frequency')
plt.show()

# Model Training

### K-means clustering

In [ ]:
#DEFINE THE NUMBER OF CLUSTERS
n_clusters = 3

In [ ]:
#CREATE THE KMEANS MODEL
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

In [ ]:
#FIT THE MODEL
kmeans.fit(X)

In [ ]:
#PREDICT THE CLUSTERS
data['cluster'] = kmeans.predict(X)

### DBSCAN Clustering

### Divisive hierarchical clustering

### Agglomerative hierarchical clustering

### Mean Shift Clustering

### Spectral Clustering

### Affinity Propagation

### BIRCH Clustering

# Transfer Learning

# Model Evaluation

# Deployment

# Monitoring